In [2]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [3]:
def extract_frames_from_video(video_path, frame_count=10):
    """
    Extrai um número específico de frames de um vídeo.
    
    :param video_path: Caminho para o vídeo.
    :param frame_count: Número de frames a serem extraídos.
    :return: Lista de frames extraídos.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

In [4]:
def load_video_dataset(data_dir, frame_count=5):
    """
    Carrega o conjunto de dados de vídeos, extrai frames e atribui rótulos.
    
    :param data_dir: Diretório raiz contendo as subpastas dos vídeos.
    :param frame_count: Número de frames a serem extraídos de cada vídeo.
    :return: Arrays de dados e rótulos.
    """
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):  # Supondo que os vídeos são arquivos .mp4
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                for frame in frames:
                    X.append(frame)
                    y.append(value)
    
    return X, y

In [5]:
def preprocess_image_without_sobel(image, target_size=(256, 256)):
    """
    Preprocess a single image for model prediction without applying the Sobel filter.

    :param image: Image data as a NumPy array.
    :param target_size: Target size for the processed image.
    :return: Preprocessed image ready for prediction.
    """
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the image
    resized_image = cv2.resize(gray_image, target_size)

    # Normalize the image
    normalized_image = resized_image / 255.0

    # Expand dimensions to match model input
    processed_image = np.expand_dims(normalized_image, axis=-1)

    return processed_image

In [6]:
def process_in_batches_no_sobel(X, batch_size=100, target_size=(256, 256)):
    """
    Processes data in batches to reduce memory consumption, without applying the Sobel filter.

    :param X: List of images to be processed.
    :param batch_size: Size of each batch.
    :param target_size: Target size for the processed images.
    :return: Processed data as a NumPy array.
    """
    X_processed = []
    
    # Divide data into batches and process each batch
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        
        # Preprocess each image in the batch
        batch_processed = [preprocess_image_without_sobel(frame, target_size) for frame in batch]
        
        X_processed.extend(batch_processed)
        
    return np.array(X_processed)

In [7]:
def build_cnn(input_shape):
    """
    Constroi o modelo de CNN.
    
    :param input_shape: Forma da entrada do modelo.
    :return: Modelo CNN compilado.
    """
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Carregar o conjunto de dados de vídeos
data_dir = 'Celeb-DF'
X, y = load_video_dataset(data_dir)

In [9]:
X_no_sobel = process_in_batches_no_sobel(X, batch_size=100, target_size=(256, 256))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_no_sobel, y, test_size=0.2, random_state=42)

In [10]:
cnn_model_no_sobel = build_cnn(input_shape=(256, 256, 1))
cnn_model_no_sobel.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_test, np.array(y_test)))

c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 94s 531ms/step - accuracy: 0.6818 - loss: 0.7281 - val_accuracy: 0.8399 - val_loss: 0.4679
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 91s 522ms/step - accuracy: 0.8477 - loss: 0.4294 - val_accuracy: 0.8702 - val_loss: 0.4187
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 88s 508ms/step - accuracy: 0.8616 - loss: 0.3877 - val_accuracy: 0.8681 - val_loss: 0.3915
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 87s 501ms/step - accuracy: 0.8746 - loss: 0.3545 - val_accuracy: 0.8760 - val_loss: 0.3905
Epoch 5/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 87s 502ms/step - accuracy: 0.8754 - loss: 0.3387 - val_accuracy: 0.8695 - val_loss: 0.3940
Epoch 6/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 89s 510ms/step - accuracy: 0.8734 - loss: 0.3358 - val_accuracy: 0.8767 - val_loss: 0.4013
Epoch 7/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 88s 506ms/step - accuracy: 0.8775 - loss: 0.3323 - val_accuracy: 0.8709 - val_loss: 0.3961
Epoch 8/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 88s 504ms/step - accuracy: 0.8852 - loss: 0

In [11]:
cnn_model_no_sobel.save('Model/meu_modelo_Nosobel.h5')